In [ ]:
%pwd

In [ ]:
# !nvidia-smi

In [ ]:
# dependencies は初回のみ実行
# %pip install ipywidgets bitsandbytes peft pyzmq transformers trl datasets sentencepiece accelerate wandb huggingface_hub argilla python-dotenv

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
# cache系は必ずteam storageへ
# TEAM_DATASETS_CACHE_DIR="/persistentshare/storage/team_kumagai/datasets"
TEAM_DATASETS_CACHE_DIR = "./.cache"

In [ ]:
import json
import os
import sys
from datetime import datetime
import logging
import random

import numpy as np
import pandas as pd

import wandb
from huggingface_hub import login, whoami

import argilla as rg

from datasets import load_dataset, Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoConfig,
    AutoModel,
    set_seed,
    Seq2SeqTrainer,
    BitsAndBytesConfig,
    LlamaTokenizer,
    TrainerCallback,
)

from transformers import TrainingArguments
from trl import DPOTrainer

import torch.distributed as dist
import multiprocessing as mp

import torch
import transformers

from typing import Any


logger = logging.getLogger()
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info("start logging...!")

In [ ]:
login(token=os.getenv("HF_TOKEN"))

In [ ]:
run = wandb.init(
    project=os.getenv("WANDB_PROJECT"),
    entity=os.getenv("WANDB_ENTITY"),
)

In [ ]:
rg.init(
    api_url=os.getenv("RG_API_URL"),
    api_key=os.getenv("RG_API_KEY"),
    workspace=os.getenv("RG_WORKSPACE"),
)

In [ ]:
# admin userならば、以下のコードでユーザー一覧を取得できる
# rg.User.list()

In [ ]:
rg.Workspace.list()

In [ ]:
workspace = rg.Workspace.from_name(os.getenv("RG_WORKSPACE"))
workspace

In [ ]:
# llm-jpの翻訳版 hh-rlhf を使う
# ライセンスはMIT
# https://huggingface.co/datasets/llm-jp/hh-rlhf-12k-ja

In [ ]:
dataset_use = load_dataset("llm-jp/hh-rlhf-12k-ja", cache_dir=TEAM_DATASETS_CACHE_DIR)
dataset_use

In [ ]:
# sourceのuniqueな値を取得
source = dataset_use["train"]["source"]
# setでユニークな値を取得
source_set = set(source)
source_set

In [ ]:
# 1件目を確認
dataset_use["train"][0]

In [ ]:
# 10件目を確認
dataset_use["train"][10 - 1]

In [ ]:
# 1500件目を確認
dataset_use["train"][1500 - 1]

In [ ]:
# 10000件目を確認
dataset_use["train"][10000 - 1]

In [ ]:
# 12000件目のデータを確認
dataset_use["train"][12000 - 1]

In [ ]:
def data_to_dpo_format(data: dict[str, Any]):
    # content: str
    # role: user or assistant
    # の構成に直す
    conversations: list[dict[str, str]] = data["conversations"]

    replace: list[dict[str, str]] = []
    # from: human or gpt
    # value: str
    # で構成されるので分離するが、contentが消えている？
    for conversation in conversations:
        str_value = conversation["value"]

        # fromをroleに変換
        str_from = conversation["from"]
        if str_from == "human":
            str_from = "user"
        elif str_from == "gpt":
            str_from = "assistant"
        else:
            logger.error(f"from is invalid: {str_from}")
            raise ValueError("unreachable")

        replace.append(
            {
                "content": str_value.strip(),
                "role": str_from,
            }
        )

    chosen: str = data["chosen"]
    rejected: str = data["rejected"]
    chosen_set = replace.copy()
    rejected_set = replace.copy()

    # 最後が user の場合は、新しく assistant を追加する
    # 最後が assistant の場合は、assistant の content に追加する
    chosen_last = chosen_set[-1]
    if chosen_last["role"] == "user":
        chosen_set.append(
            {
                "content": chosen.strip(),
                "role": "assistant",
            }
        )
    else:
        chosen_last["content"] += "" + chosen.strip()

    rejected_last = rejected_set[-1]
    if rejected_last["role"] == "user":
        rejected_set.append(
            {
                "content": rejected.strip(),
                "role": "assistant",
            }
        )
    else:
        rejected_last["content"] += "" + rejected.strip()

    # 他はmetadataとして追加
    source: str = data["source"]

    return {
        "chosen": chosen_set,
        "rejected": rejected_set,
        "metadata": {
            "source": source,
        },
    }


def dpo_format_to_field(data: list[dict[str, str]]):
    # role: user or assistant
    # content: str
    # をfieldの表示形式に変換する
    ret = ""
    for item in data:
        role = item["role"]
        content = item["content"]
        ret += f"{role}: {content}\n"

    return ret

In [ ]:
test1 = data_to_dpo_format(dataset_use["train"][10000 - 1])
test1

In [ ]:
print(dpo_format_to_field(test1["chosen"]))

In [ ]:
# 全部を変換する
train_dataset_use = dataset_use["train"].map(data_to_dpo_format)
# sourceとconversationsのkeyは消す
train_dataset_use = train_dataset_use.remove_columns(["source", "conversations"])

In [ ]:
train_dataset_use[0]

In [ ]:
train_dataset_use[10000 - 1]

In [ ]:
def get_fields():
    return [
        rg.TextField(name="chosen"),
        rg.TextField(name="rejected"),
    ]


def get_questions():
    return [
        rg.RatingQuestion(
            name="chosen_rating",
            values=[1, 2, 3, 4, 5],
            required=True,
        ),
        rg.RatingQuestion(
            name="rejected_rating",
            values=[1, 2, 3, 4, 5],
            required=True,
        ),
        rg.TextQuestion(
            name="modify_chosen",
            required=False,
        ),
        rg.TextQuestion(
            name="modify_rejected",
            required=False,
        ),
    ]


def get_metadata_properties():
    return [
        rg.TermsMetadataProperty(
            name="source",
            values=[
                "harmless-base",
                "helpful-base",
                "helpful-online",
                "helpful-rejection-sampled",
            ],
        ),
    ]

In [ ]:
def to_record(data: dict[str, str]):
    return rg.FeedbackRecord(
        fields={
            "chosen": dpo_format_to_field(data["chosen"]),
            "rejected": dpo_format_to_field(data["rejected"]),
        },
        # TODO: suggestionsでmodify_chosenとmodify_rejectedを追加する
        # suggestions=[
        #     {
        #         "question_name": "modify_chosen",
        #         "value": "",
        #     }
        # ]
        metadata=data["metadata"],
    )

In [ ]:
to_record(train_dataset_use[0])

In [ ]:
def create_rg_dataset(dataset_use):
    rg_dataset = rg.FeedbackDataset(
        guidelines="hh-rlhfのデータセットを使って、chosenとrejectedを評価してください。必要があれば最後のassistantの文章のみを修正してください。",
        fields=get_fields(),
        questions=get_questions(),
        metadata_properties=get_metadata_properties(),
    )
    # to_recordを使って全て変換する
    rg_records = [to_record(data) for data in dataset_use]
    
    # ログで確認
    logger.info(f"rg_records: {rg_records}")
    
    rg_dataset.add_records(rg_records)
    
    return rg_dataset

In [ ]:
rg_dataset = create_rg_dataset(train_dataset_use)
# pushする
# remote_rg_dataset = rg_dataset.push_to_argilla(
#     name="second-hh-rlhf-12k-ja", workspace=workspace, show_progress=True
# )

In [ ]:
# remote_rg_dataset を hf にpushする
# remote_rg_dataset.push_to_huggingface(
#     "hironow/test-hh-rlhf-12k-ja", generate_card=True,
# )